# Script para ler o Diário Oficial do Município de São Paulo

## Script BASE

### Importando as bibliotecas

In [1]:
from datetime import date, datetime, timedelta
import os
import fitz as fz
import tabula

### Obter data atual

In [2]:
def get_date_today():
    '''Obtém a data atual e retorna uma string: (YYYY-mm-dd)'''
    dt_now = date.today()
    if dt_now.weekday() == 0:
            dt_now = dt_now + timedelta(-2)
            dt_now = dt_now.strftime("%Y-%m-%d")
    else:
            dt_now = dt_now.strftime("%Y-%m-%d")
    
    return dt_now

dt_now = get_date_today()

### Obtendo diretórios

In [3]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        
        print(f'### INFO:  DIRS ###')
        print(f'BASE_DIR:  {BASE_DIR}')
        print(f'DATA_DIR:  {DATA_DIR}')
        print(f'SAVE_DIR:  {SAVE_DIR}')
        print(f'DATA_FILE  {DATA_FILE}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()

### INFO:  DIRS ###
BASE_DIR:  c:\Projetos\scrapy_dom
DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-26
SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-26
DATA_FILE  c:\Projetos\scrapy_dom\outputs_files\2022-05-26\DOM_SP-67_98_111_2022-05-26.tsv


### Obtendo Informações do DOM

In [107]:
def get_info_dom():
        '''Obtém as informações do DOM.
                return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM        
        '''
        dt_now = get_date_today()
        BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()
        YEAR_DOM = date.today().year - 1955
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' ('

        with fz.open(DATA_DIR +'\\'+ DATA_FILES[3]) as file:
                search_edition = ''
                for page in file:
                        search_edition += page.get_text()

        search_edition = search_edition[search_edition.find(HEAD_DOM):]
        EDITION_DOM = search_edition[search_edition.find('(')+1:search_edition.find(')')]
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' (' + EDITION_DOM + ')'
        PAGS_DOM = str(len(DATA_FILES))
        DOM = f'DOM_SP-{str(YEAR_DOM)}_{str(EDITION_DOM)}_{PAGS_DOM}_{str(dt_now)}'

        print('### INFO: DOM-SP ###')
        print(f'DATA: {dt_now}')
        print(f'PAGS: {PAGS_DOM}')
        print(f'HEAD: {HEAD_DOM}')
        print(f' DOM: {DOM}')

        

        return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM

YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM = get_info_dom()

### INFO:  DIRS ###
BASE_DIR:  c:\Projetos\scrapy_dom
DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-26
SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-26
DATA_FILE  c:\Projetos\scrapy_dom\outputs_files\2022-05-26\DOM_SP-67_98_111_2022-05-26.tsv
### INFO: DOM-SP ###
DATA: 2022-05-26
PAGS: 111
HEAD: São Paulo, 67 (98)
 DOM: DOM_SP-67_98_111_2022-05-26


### Salva arquivo final

In [108]:
def save_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","w",encoding='utf8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

### Obtem contéudo do arquivo TSV

In [109]:
with open(DATA_FILE,'r') as file:
    conteudo = file.read()

## Scripts de Listagens

### Seção SERVIDORES

In [148]:
# Localiza o início das seções SERVIDORES e CONCURSOS
sc_servidores = conteudo.find('\nSERVIDORES\n')
sc_concursos = conteudo.find('\nCONCURSOS\n')
print(f"SERVIRODRES: {sc_servidores} / CONCURSOS: {sc_concursos}")
conteudo_servidores = conteudo[sc_servidores:sc_concursos]

SERVIRODRES: 770850 / CONCURSOS: 1044652


#### Limpeza dos dados

In [159]:
conteudo_servidores = conteudo_servidores.replace('\t',' ')
conteudo_servidores = conteudo_servidores.replace('""','')
conteudo_servidores = conteudo_servidores.replace(' \n','\n')
conteudo_servidores = conteudo_servidores.replace('\n ','\n')
conteudo_servidores = conteudo_servidores.replace('    ',' ')
conteudo_servidores = conteudo_servidores.replace('   ',' ')
conteudo_servidores = conteudo_servidores.replace('  ',' ')
conteudo_servidores = conteudo_servidores.replace('-\n','')
conteudo_servidores = conteudo_servidores.replace(',\n',', ')
conteudo_servidores = conteudo_servidores.replace(', \n',', ')
conteudo_servidores = conteudo_servidores.replace('/\n','/')
conteudo_servidores = conteudo_servidores.upper()
conteudo_servidores = conteudo_servidores.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_servidores = conteudo_servidores.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_servidores = conteudo_servidores.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_servidores = conteudo_servidores.replace('COORDENADORIA DE ADMINISTRAÇÃO\nE FINANÇAS','COORDENADORIA DE ADMINISTRAÇÃO E FINANÇAS')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_servidores = conteudo_servidores.replace('RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO\nDEFERIDO','RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_servidores = conteudo_servidores.replace('ACI-\nDENTE','ACIDENTE')
conteudo_servidores = conteudo_servidores.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_servidores = conteudo_servidores.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_servidores = conteudo_servidores.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_servidores = conteudo_servidores.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_servidores = conteudo_servidores.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_servidores = conteudo_servidores.replace('DECRETO NO\n','DECRETO NO')
conteudo_servidores = conteudo_servidores.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_servidores = conteudo_servidores.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_servidores = conteudo_servidores.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_servidores = conteudo_servidores.replace('PORTARIA\n','PORTARIA ')
conteudo_servidores = conteudo_servidores.replace(' DOC\nSEI ',' DOC SEI ')
conteudo_servidores = conteudo_servidores.replace(' DA LEI\n',' DA LEI ')
conteudo_servidores = conteudo_servidores.replace('\nAOS ',' AOS ')
conteudo_servidores = conteudo_servidores.replace(' SÃO\n',' SÃO ')
conteudo_servidores = conteudo_servidores.replace(' À\n',' À ')
conteudo_servidores = conteudo_servidores.replace(' ÀS\n',' ÀS ')
conteudo_servidores = conteudo_servidores.replace(' AS\n',' AS ')
conteudo_servidores = conteudo_servidores.replace(' E\n',' E ')
conteudo_servidores = conteudo_servidores.replace(' O\n',' O ')
conteudo_servidores = conteudo_servidores.replace(' DA\n',' DA ')
conteudo_servidores = conteudo_servidores.replace('\nDA ',' DA ')
conteudo_servidores = conteudo_servidores.replace(' DAS\n',' DAS ')
conteudo_servidores = conteudo_servidores.replace(' DA(S)\n',' DA(S) ')
conteudo_servidores = conteudo_servidores.replace(' DE\n',' DE ')
conteudo_servidores = conteudo_servidores.replace('\nDE ',' DE ')
conteudo_servidores = conteudo_servidores.replace(' DO\n',' DO ')
conteudo_servidores = conteudo_servidores.replace(' DOS\n',' DOS ')
conteudo_servidores = conteudo_servidores.replace(' DO(S)\n',' DO(S) ')
conteudo_servidores = conteudo_servidores.replace(' COM\n',' COM ')
conteudo_servidores = conteudo_servidores.replace('\nCOM ',' COM ')
conteudo_servidores = conteudo_servidores.replace(' EM\n',' EM ')
conteudo_servidores = conteudo_servidores.replace(' PARA\n',' PARA ')
conteudo_servidores = conteudo_servidores.replace(' POR\n',' POR ')
conteudo_servidores = conteudo_servidores.replace(' NA\n',' NA ')
conteudo_servidores = conteudo_servidores.replace(' NAS\n',' NAS ')
conteudo_servidores = conteudo_servidores.replace(' NO\n',' NO ')
conteudo_servidores = conteudo_servidores.replace(' NOS\n',' NOS ')
conteudo_servidores = conteudo_servidores.replace('A PARTIR DE',' A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
conteudo_servidores = conteudo_servidores.replace('REG. FUNC. NOME CONTROLE NO ','RF NOME CONTROLE_NO\n')
conteudo_servidores = conteudo_servidores.replace('R.F./V NOME DIAS/EXERCÍCIO A_PARTIR_DE','RF/V NOME DIAS/EXERCÍCIO A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('REG. FUNC/VINC NOME NÍVEL CAT. SÍMBOLO A_PARTIR_DE','RF/v NOME NÍVEL_CAT SÍMBOLO A_PARTIR_DE')
conteudo_servidores = conteudo_servidores.replace('RF V NOME EH A PARTIR MOTIVO','RF VINC NOME EH A_PARTIR MOTIVO')

In [160]:
#print(conteudo_servidores)
save_file(SAVE_DIR,conteudo_servidores,'seção_servidores')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\seção_servidores.txt'

In [156]:
#print(conteudo_servidores)

### Relação de Licença Médica

#### CONTEÚDO PRINCIPAL

In [157]:
conteudo_licenca_medica = conteudo_servidores

#sc_lm = conteudo_licenca_medica.find('\nRELAÇÃO DE LICENÇA MÉDICA')
#conteudo_licenca_medica = conteudo_licenca_medica[sc_lm:]
#sc_lm

In [161]:
i = 0
#rl_lm =  conteudo_licenca_medica.split('\n')
rl_lm =  conteudo_servidores.split('\n')
licencas = {}
listas = []

for line in rl_lm:
    if 'LICENÇAS MÉDICAS' in line and not 'SEÇÃO DE LICENÇAS MÉDICAS' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    elif 'RELAÇÃO DE' in line and not 'RELAÇÃO DE LICENÇA MÉDICA' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    
    i += 1

print(licencas)
print(listas)
print(f'\nTotal de listas: {len(listas)}')
save_file(SAVE_DIR,str(licencas),'listagens_dom')

{'RELAÇÃO DE SERVIDORES APROVADOS NO “CURSO DE APERFEIÇOAMENTO PARA INSPETORES DO NÍVEL III – MÓDULO EAD”, REALIZADO NO MODO EAD, CONFORME SEGUE:': 8, 'RELAÇÃO DE SERVIDORES CONVOCADOS PARA PERÍCIA DE LICENÇA': 294, 'RELAÇÃO DE SERVIDORES CONVOCADOS PARA AVALIAÇÃO COM MÉDICO PERITO ESPECIALISTA': 470, 'RELAÇÃO DE SERVIDORES CONVOCADOS PARA RETORNO NA AVALIAÇÃO MÉDICA DE ACIDENTE DE TRABALHO': 483, 'RELAÇÃO DE CONVOCADOS PARA PERÍCIA MÉDICA DE ISENÇÃO DE IMPOSTO DE RENDA': 521, 'RELAÇÃO DE CONVOCADOS PARA JUNTA DE AVALIAÇÃO DE INCAPACIDADE PERMANENTE PARA O TRABALHO': 530, 'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79': 548, 'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO': 937, 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA': 953, 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA': 955, 'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA': 969, 'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORA

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\listagens_dom.txt'

#### RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79

In [127]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [128]:
'''Listagem: RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [129]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79

In [130]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#lm_8989_negada


In [131]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        nome = str(' ').join(s_item[1:-3])
        #print(f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}\\n')
        str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
    i += 1

In [132]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NEGADAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE

In [133]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#lm_faltas_pensao

In [134]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;A_PARTIR_DE;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 3 and s_item[0][0].isnumeric():
        if s_item[-1][0].isnumeric():
            nome = str(' ').join(s_item[1:-1])
            str_list_items += f'{s_item[0]};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    i += 1

In [135]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_PARA_JUNTA_MÉDICA_PARA_FINS_DE_PENSÃO_POR_MORTE.csv'

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [136]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [137]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric():
        if s_item[-3].isnumeric():
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}/{DOM}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    i += 1

In [138]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_FALTA_DO_SERVIDOR_À_PERÍCIA.csv'

#### LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO

In [162]:
RELACAO = 'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO',
 'SECRETARIA MUNICIPAL DA SAUDE',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '7196172/1 DEBORA MENDES DA SILVA 007 08/01/2022 143',
 '7400799/1 FERNANDA FERNANDES BARBALHO 003 29/12/2021 143',
 'SECRETARIA MUNICIPAL DE EDUCACAO',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '7371799/1 MARCIA VALERIA ALVES DA COSTA 010 08/11/2021 143',
 '7437731/1 JOANA OLHER DA SILVA 002 13/09/2021 143',
 '8209928/1 ROSANA DE CASSIA BARON TELLES 008 11/11/2021 143',
 '8209928/2 ROSANA DE CASSIA BARON TELLES 008 11/11/2021 143',
 '8280444/2 ROBERTA OLIVEIRA CORREIA 007 23/08/2021 146',
 '8280444/3 ROBERTA OLIVEIRA CORREIA 007 23/08/2021 146',
 '8792607/1 MARCIA ANDRADE DA COSTA 001 20/12/2021 143',
 'DIVISÃO DE PERÍCIA MÉDICA - COGESS',
 'SEÇÃO DE LICENÇAS MÉDICAS']

In [165]:
'''Listagem: LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [166]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\LICENÇAS_MÉDICAS_CONCEDIDAS_NEGADAS_EM_GRAU_DE_RECURSO.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA

In [167]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA',
 '6534368/ CHRISTIANO JOSE BEZERRA DA SILVA 24/05/2022']

In [172]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DATA;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [173]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_PARA_AVALIAÇÃO_DE_ESPECIALISTA.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA

In [177]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA',
 '7325444/ SERGIO MARTINS RSTON 24/05/2022',
 'DIVISÃO DE PERÍCIA MÉDICA - COGESS',
 'SEÇÃO DE LICENÇAS MÉDICAS']

In [178]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DATA;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [179]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_P__AVALIAÇÃO_DE_ISENÇÃO_DE_IMPOSTO_DE_RENDA.csv'

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [180]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [183]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-2]
        4. A_PARTIR_DE  : date()     s_item[-2]
        5. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [184]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_FALTA_DO_SERVIDOR_À_PERÍCIA.csv'

#### RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO

In [185]:
RELACAO = 'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [190]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.split(' ')
    if 'RF NOME' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [191]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-26\\RELAÇÃO_DE_SERVIDORES_SUBMETIDOS_À_AVALIAÇÃO_DE_CAPACIDADE_LABORATIVA_COM_CAPACIDADE_LABORATIVA_PARA_A_SUA_FUNÇÃO.csv'